## Extracting environmental Features

In [ ]:
station_environmental_data=pd.read_csv('./TARA_registies_water_column.tab',delimiter='\t',skiprows=2542)

In [ ]:
def redefine_depth_code(string):
    if '[SRF]' in string:
        output='SUR'
    elif '[MES]' in string:
        output='MES'
    elif '[DCM]' in string:
        output='DCM'
    elif '[MIX]' in string:
        output='MXL'
    elif '[ZZZ]' in string:
        output='ZZZ'
    else:
        output='NA'
    return output

In [ ]:
station_environmental_data['updated_station']=station_environmental_data['Station (TARA_station#)'].str.replace('TARA_','Station')
station_environmental_data['updated_station']=station_environmental_data['updated_station'].str.replace('n0','n')
station_environmental_data['updated_depth_code']=station_environmental_data['Env feature ([abbreviation] description (E...)'].apply(redefine_depth_code)

In [ ]:
def depth_parsing_function(string):
    if string.lower().islower():
        mean_depth='NA'
    elif '-' in string:
        min_depth=string.split('-')[0]
        max_depth=string.split('-')[1]
        mean_depth=np.mean([float(min_depth),float(max_depth)])
    else:
        mean_depth=float(string)
    return(mean_depth)

In [ ]:
def lat_long_matching_function(row):
    sample_slice=station_environmental_data[(station_environmental_data['updated_station']==row['station']) & (station_environmental_data['updated_depth_code']==row['depth_class'])]
    lat=np.mean(sample_slice['Latitude (TARA_event-datetime_station#_...)'])
    lon=np.mean(sample_slice['Longitude (TARA_event-datetime_station#_...)'])
    row['depth']=np.mean(sample_slice['Depth, nominal (of the environmental feature)'].apply(float))
    row['lat']=lat
    row['lon']=lon
    return row

In [ ]:
station_environmental_data['Latitude (TARA_event-datetime_station#_...)']=station_environmental_data['Latitude (TARA_event-datetime_station#_...)'].apply(float)
station_environmental_data['Longitude (TARA_event-datetime_station#_...)']=station_environmental_data['Longitude (TARA_event-datetime_station#_...)'].apply(float)
station_environmental_data['Depth, nominal (of the environmental feature)']=station_environmental_data['Depth, nominal (of the environmental feature)'].apply(depth_parsing_function)

In [ ]:
genome_covariates.to_csv('station_environmental_data.csv', index=False)